In [76]:
# Copy and paste latest sbhandler.py code

# from sbhandler import *
from math import pow    
# from fastai.vision.all import *

# Needed imports
import numpy as np
import h5py

STOP_CODON = np.array([84, 65, 65, 84, 65, 71]) # T A A T A G

# Source code for functions from seqblock_handler.py
def load_matlab_file(mat_file, variable_name): # Note the double return statement
    """_summary_
    //UNIQUE: Only one variable for this .mat file
    
    Args:
        mat_file (string): Name of .mat file to intake (path must be given if not in current dir)
        variable_name (string): which variable from the mat file. Only one here because of the .mat file structure.

    Returns:
        h5py_object, data: Overall h5py_object and the specific variable as an nparray
    """
    h5py_object = h5py.File(mat_file, 'r')
    data = h5py_object.get(variable_name)
    data = np.array(data)
    return h5py_object, data
class DNA_SeqBlocks():
    """Mat dataset 
    Loads the mat file with DNA sequences for easy iteration through all stored DNA sequence lists of varying coverage
    """
    def __init__(self, h5py_object, data):
        """Initialize DNA_SeqBlocks() object
        Args:
            h5py_object (h5py_object): first return variable of load_matlab_file
            data (np.array): second return variable of load_matlab_file
        """
        self.h5py_object = h5py_object
        self.data = np.transpose(data)
        self.size = len(data) 
        
    def get_seqblock(self, number, true_len = 0, df=False): # checked
        if true_len != 0:
            x = self.h5py_object[self.data[0, number-1]][0:true_len,:]
            if df == True:
                return x
            return np.transpose(x) # no need to tranpose if its for pandas
            
        x = self.h5py_object[self.data[0, number-1]][:, :] # //UNIQUE: 0th column because .mat file, align3 variable only has one row
        if df == True:
            return x
        return np.transpose(x)
    
    def get_target(self, number):
        x = self.h5py_object[self.data[0, number-1]][:, 0]
        return np.transpose(x)
    
    def get_coverage_count(self, number):
        """Coverage count

        Args:
            number (int): which sequence? matlab idx

        Returns:
            int: Number of reads
        """
        x = self.h5py_object[self.data[0, number-1]][0,:]
        coverage_count = int((len(x)-4)/2)
        return coverage_count
    
    def get_barcode(self, number): # //TODO: Ask Yiyang about a mutated stop codon.
        """Barcode finder
        
        Changes that need to be made: use target sequence to find stop codon (reads may have mutated stop codon like 22776). Use read that doesn't terminate in indels to find the barcode.

        Args:
            number (int): which sequence? matlab indexing

        Returns:
            barcode, true_len: returns barcode uint16 ndarray, last matlab idx of stop codon
        """
        x = self.h5py_object[self.data[0, number-1]][:, 0]
        barcode = ""
        true_len = 0
        total = x.size
        
        idx = 0
        search_stop=True
        
        # This block is concise but had 2x the runtime
        # while idx < total and search_stop: # To identify the stop codon
        #     if (np.all(x[idx:idx+6] == STOP_CODON)):  # Translates to 'TAATAG'
        #         true_len = idx+6
        #         search_stop = False
        #     idx += 1
        
        while idx < total and search_stop:
            if (x[idx] == 84):
                try:
                    if (x[idx+1] == 65):
                        if x[idx+2] == 65:
                            if x[idx+3] == 84:
                                if x[idx+4] == 65:
                                    if x[idx+5] == 71:
                                        true_len = idx+6
                                        search_stop = False
                except: 
                    barcode = None
                    true_len = 0
                    return barcode, true_len
            idx += 1
        barcode = x[true_len:total]
            
        return barcode, true_len
    
    def get_nuc_qscores(self, number, nuc_pos, coverage_count=0): # checked
        if coverage_count == 0:
            cov = self.get_coverage_count(number)
            x = self.h5py_object[self.data[0, number-1]][nuc_pos-1, (1+cov):(1+2*cov)]
            return np.transpose(x)
        else:
            x = self.h5py_object[self.data[0, number-1]][nuc_pos-1, (1+coverage_count):(1+2*coverage_count)]
            return np.transpose(x)
    
    def get_nuc_reads(self, number, nuc_pos, coverage_count=0): # checked
        if coverage_count == 0:
            x = (self.h5py_object[self.data[0,number-1]][nuc_pos-1, 1:self.get_coverage_count(number)+1])
            return np.transpose(x)
        else: 
            x = self.h5py_object[self.data[0,number-1]][nuc_pos-1, 1:(coverage_count+1)]
            return np.transpose(x)
    
    def get_reads(self, number, coverage_count=0, true_len=0): # CHECK THE INDEXING
        if coverage_count == 0 and true_len == 0:
            _, true = self.get_barcode(number)
            x = (self.h5py_object[self.data[0,number-1]][0:true, 1:self.get_coverage_count(number)+1])
            return np.transpose(x)
        else: 
            x = self.h5py_object[self.data[0,number-1]][0:true_len, 1:(coverage_count+1)]
            return np.transpose(x)
    
    def get_qscores(self, number, coverage_count=0, true_len=0):
        if coverage_count == 0 and true_len == 0:
            _, true = self.get_barcode(number)
            cov = self.get_coverage_count(number)
            x = (self.h5py_object[self.data[0,number-1]][0:true, (1+cov):(1+2*cov)])
            return np.transpose(x)
        else: 
            x = self.h5py_object[self.data[0,number-1]][0:true_len, 1:(coverage_count+1)]
            return np.transpose(x)        
    
    def get_interp_changes(self, number, coverage_count=0, true_len=0): # checked
        if coverage_count == 0 and true_len == 0:
            _, true = self.get_barcode(number)
            x = self.h5py_object[self.data[0,number-1]][0:true,(1+2*self.get_coverage_count(number))]
            return np.transpose(x)
        else:
            x = self.h5py_object[self.data[0,number-1]][0:true_len,(1+2*coverage_count)]
            return np.transpose(x)
    
    def get_interp_mutations(self, number, coverage_count=0, true_len=0):
        if coverage_count == 0 and true_len == 0:
            _,true=self.get_barcode(number)
            x = self.h5py_object[self.data[0,number-1]][0:true,(3+2*self.get_coverage_count(number))]
            return np.transpose(x)
        else:
            x = self.h5py_object[self.data[0,number-1]][0:true_len,(3+2*coverage_count)]     
            return np.transpose(x)      
    
    def get_interp_consensus(self, number, coverage_count=0, true_len=0):
        if coverage_count == 0 and true_len == 0:
            _, true= self.get_barcode(number)
            x = self.h5py_object[self.data[0,number-1]][0:true,(2+2*self.get_coverage_count(number))]
            return np.transpose(x) 
        else:
            x = self.h5py_object[self.data[0,number-1]][0:true_len,(2+2*coverage_count)]
            return np.transpose(x) 
    
def seqblock_parser(seqblock):
    seqblock_parsed = CleanSeqBlock()
    total_columns = len(np.transpose(seqblock)) # is there a more efficient way?
    total_rows = len(seqblock)
    number_of_reads = int((total_rows - 4)/2) # last three rows have intreptations, first row is non-mutated target sequence, and N quality scores for N reads
    
    seqblock_parsed.cov = number_of_reads;
    seqblock_parsed.len = total_columns;
    
    # print(total_columns)
    # print(total_rows)
    tag = ""
    tag_length = 10_000
    
    for i in range(total_rows):
        # print(f" The sequenceblock input {seqblock}") #debug
        # print(f" The m x n size of the seqblock {seqblock.shape}") #debug
        # print(f" The ith row of the seqblock {current_seq}") #debug
        # print(f" The m x n size of the sequence {current_seq.shape}") #debug
        
        # new_format = np.chararray(total_columns) //CHANGED to string
        stop_found = False
        # new_format[:] = 'q' #debug
        
        # print(f" The 0th entry of the initialized char array {new_format[0, 0]}") #debug
        # print(f" Its row size {len(new_format[0])}")
        # if (i == 0):
        #     original = ""
        #     for j in range(total_columns):
        #         original += chr(seqblock[i,j])
                
                
        
        # if (i != 0):
  
        if (i == 0 or i == 1):
            new_format = ""
            j=0
        
            while j < total_columns:
                # print(j)
                                        
                if stop_found is False:
                    if (seqblock[i,j] == ord('T') and (j+6) < total_columns):
                        if (seqblock[i,j+1] == ord('A') and seqblock[i, j+2] == ord('A') and seqblock[i, j+3] == ord('T') and seqblock[i, j+4] == ord('A') and seqblock[i, j+5] == ord('G')):
                            new_format += 'T'
                            new_format += 'A'
                            new_format += 'A'
                            new_format += 'T'
                            new_format += 'G'
                            j = j+6
                            stop_found = True
                    if stop_found is False:
                        new_format += chr(seqblock[i,j])
                        j+=1
            
                elif stop_found is True and i == 1:
                    tag += chr(seqblock[i,j])
                    j+=1
                else:
                    j = total_columns
            if (i == 1): 
                seqblock_parsed.barcode = tag
                tag_length = len(tag)
        else:
            new_format = ""
            for j in range(total_columns-tag_length):
                new_format += chr(seqblock[i,j])
                
                
        if (i == 0):
            seqblock_parsed.target = new_format
        elif (i == total_rows - 1): # last row is mutations 'x', total_rows is one more than total index
            seqblock_parsed.interp_mutations = new_format
        elif (i == total_rows - 2): # 2nd last is subjective consensus
            seqblock_parsed.interp_consensus = new_format
        elif (i == total_rows - 3): 
            seqblock_parsed.interp_changes = new_format
        elif (i > 0 and i < number_of_reads + 1):
            seqblock_parsed.reads.append(new_format)
        else:
            seqblock_parsed.qscores.append(new_format)
    return seqblock_parsed     
class CleanSeqBlock():
    def __init__(self):
        self.cov, self.len, self.target, self.interp_changes, self.interp_mutations, self.interp_consensus, self.barcode = 0,0,0,0,0,0,0
        self.reads, self.qscores = [], []

def pscore(Qscore): 
    return pow(10, -(Qscore-33)/10)

In [77]:

# load_matlab_file function Testing
file_name = 'consensus1.mat'
variable_of_interest = 'align3'
h5py_object, data = load_matlab_file(file_name, variable_name=variable_of_interest)

# DNA_seqs can be used to grab each raw_seqblock
DNA_seqs = DNA_SeqBlocks(h5py_object=h5py_object, data=data) 

FILE_SIZE = DNA_seqs.size #Constant

In [ ]:
# Load in barcodes and truelens for the day
import pickle
df = pickle.load(open('pickle/v1BarcodesTruelen.pickle', 'rb'))
BARCODES = df.barcode
TRUELENS = df.true_len

## Feature: Using pandas dataframes and saving it in pickle. 
- The main dataset won't ever change

In [83]:
# Make sure it works on a small dataset first.
tinyDS = range(1, 10001)
DS = range(1, FILE_SIZE+1)

In [81]:
# Rename each column for readability
def rename(df, transpose=False):
    if transpose == True: df = df.T
    cov = int(0.5*(df.columns.size-4))
    based_on_cov = ['target']
    based_on_cov.extend([f'read{i}' for i in range(1, cov+1)])
    based_on_cov.extend(x for x in [f'q{i}' for i in range(1, cov+1)])
    based_on_cov.extend(['changes', 'contig', 'mutations'])
    df.columns = based_on_cov
    return df

In [84]:
# Testing on a single seqblock
import pandas as pd
dflist = []

# for num in tinyDS:
for num in tinyDS:
    true_len = TRUELENS[num]
    df = pd.DataFrame(DNA_seqs.get_seqblock(num, true_len=true_len, df=True))
    df = rename(df)
    df.index.name = f'SB{num}' # Used to keep track of which specific sequence block the dataframe references
    dflist.append(df)


In [86]:
# Only return the sections with a z conflict
chr = ord('z')


conflicts = []
for i in dflist:
    test = i[i.changes == chr]
    # if int(i.index.name[2:]) % 500 == 0: print(f'{test} \n \n')
    if (test.size > 0): conflicts.append(test)
print(f'\n \n')
len(conflicts), len(dflist), len(conflicts)/len(dflist)
        
        

(83, 1000, 0.083)

In [87]:
print(conflicts[0])

      target  read1  read2  read3   q1  q2   q3  changes  contig  mutations
SB13                                                                       
506       67     67     65     67  126  75  126      122      67         32


In [ ]:
# Easy fix...